In [3]:
import pandas as pd
from pandas import DataFrame

gene2go = pd.read_csv("../Datasets/NCBI/gene2go/gene2go" , "\t")
gene2go = gene2go.loc[gene2go['#tax_id'] == 9606]
gene2go = gene2go.drop('')
#humanGenome = pd.read_csv("../Datasets/NCBI/Homo_sapiens.gene_info/Homo_sapiens.gene_info", "\t")
#cancerData = pd.read_csv("../Datasets/UnitProtKB/unitprot-cancer/unitprot-cancer.txt")
gene2go

,#tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
575368,9606,1,GO:0002576,TAS,-,platelet degranulation,-,Process
575369,9606,1,GO:0003674,ND,-,molecular_function,-,Function
575370,9606,1,GO:0005576,IDA,-,extracellular region,3458201|27068509,Component
575371,9606,1,GO:0005576,TAS,-,extracellular region,-,Component
575372,9606,1,GO:0005615,HDA,-,extracellular space,16502470,Component
575373,9606,1,GO:0008150,ND,-,biological_process,-,Process
575374,9606,1,GO:0031012,IDA,-,extracellular matrix,27559042,Component
575375,9606,1,GO:0031093,TAS,-,platelet alpha granule lumen,-,Component
575376,9606,1,GO:0034774,TAS,-,secretory granule lumen,-,Component
575377,9606,1,GO:0043312,TAS,-,neutrophil degranulation,-,Process


In [7]:
# Generating the scripts

# (1) name : name of the table
# (2) lines : list of all lines to be inserted
# e.g. generateInsertScript("users", [["1", "25", "hello"], ['2', '3', '4']])
def generateInsertScript(name, lines):
    query = "INSERT INTO TABLE " + name  + " VALUES"
    for line in lines:
        first = True
        query = query + " ("
        for v in line:
            if first:
                first = False
            else:
                query = query + ", "
            query = query + str(v)
        query = query + ")"
    if len(lines) == 0:
        return ";"
    else:
        return query + ";"
generateInsertScript("users", [("1", "25", "hello"), (2, 3, 4)])

'INSERT INTO TABLE users VALUES (1, 25, hello) (2, 3, 4);'

In [10]:
# return a dataframe's columns named after array 'attributes'
# inefficient, rather slow
def extractColumns(df, attributes):
    return [df[attributes].iloc[[i]].values.tolist()[0] for i in range(len(df))]


['GO:0005634',
 'GO:0008150',
 'GO:0003677',
 'GO:0003700',
 'GO:0005634',
 'GO:0005634',
 'GO:0006351',
 'GO:0006355',
 'GO:0003674',
 'GO:0005886',
 'GO:0008150',
 'GO:0016021',
 'GO:0003674',
 'GO:0005794',
 'GO:0005794',
 'GO:0005886',
 'GO:0008150',
 'GO:0005750',
 'GO:0006122',
 'GO:0008121',
 'GO:0055114',
 'GO:1902600',
 'GO:0003674',
 'GO:0005634',
 'GO:0008150',
 'GO:0005886',
 'GO:0009507',
 'GO:0009535',
 'GO:0009567',
 'GO:0009977',
 'GO:0009977',
 'GO:0010027',
 'GO:0031361',
 'GO:0033281',
 'GO:0043235',
 'GO:0043953',
 'GO:0065002',
 'GO:0000808',
 'GO:0003677',
 'GO:0003688',
 'GO:0005515',
 'GO:0005524',
 'GO:0005634',
 'GO:0005634',
 'GO:0005664',
 'GO:0006260',
 'GO:0006270',
 'GO:0003676',
 'GO:0004386',
 'GO:0005524',
 'GO:0005634',
 'GO:0004332',
 'GO:0005739',
 'GO:0006094',
 'GO:0006096',
 'GO:0006096',
 'GO:0006098',
 'GO:0006979',
 'GO:0009507',
 'GO:0009507',
 'GO:0009507',
 'GO:0009534',
 'GO:0009536',
 'GO:0009570',
 'GO:0010287',
 'GO:0046686',
 'GO:00048